In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.8 MB/s eta 0:00:00


In [2]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 9.6 MB/s eta 0:00:00


In [3]:
import math
import numpy as np
import pandas as pd
import random
import re
import torch
from torch.utils.data import DataLoader, Dataset

from transformers import AutoTokenizer, GPT2LMHeadModel
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from pytorch_lightning import Trainer, LightningModule

In [4]:
data = pd.read_csv('Chatbot_Data.csv')
data

,Q,A,label
0,오늘 새로 온 신입사원 봤어요? 모여서 깔깔대며 웃는 모습들이 너무 부럽더라구요. ...,"맞아요, 좋을 때다 싶더라구요. 무슨 마음인지 이해가 갈 것 같아요. 저도 제가 못...",2
1,그렇죠? 저만 그런 건 아니겠죠? 오늘 해맑게 웃는 친구들을 보니 내가 너무 부정적...,"너무 행복하고 싶고, 잘 웃고 싶은데 마음처럼 되지 않으니 속상하고 슬퍼지는 거군요...",1
2,겉모습이라도 그렇게 자기 감정을 잘 표현하고 밝은 사람들을 보면 부러워요. 왜 나는...,꼭 활짝 웃고 소리 내어 웃어야만 아름다운 건 아니에요. 조용하고 따뜻한 미소도 충...,2
3,어머 그런가요? 저는 웃음이 예쁜 사람이 아니라고 생각했어요. 늘 잘 웃는 사람들을...,맞아요. 너무 자신을 과소평가하지 않아도 되어요. 대리님의 미소만으로도 충분히 위로...,2
4,저는 늘 저 자신이 어둡다고만 생각했고 명랑하고 밝은 사람들이 웃는 모습을 보면 늘...,부러운 마음은 이해가 가요. 누구나 가질 수 있는 감정이죠. 하지만 따지고 보면 이...,2
...,...,...,...
397,"저도 그런 것 같아서 용기를 내서 말했는데, 반응이 없으니까 불안해요.",사수가 표현을 잘 못하는 성격이잖아요. 아마 쑥스러워서 말 못했을 수도 있어요.,0
398,저라면 좋으면 좋다고 바로 말할 텐데 사수가 아무 말이 없으니까 거절당할까봐 두렵기...,마음을 좀 편히 갖고 기다려 봐요. 만약에 거절을 당하더라도 마음의 상처는 생기겠지...,1
399,근데 거절당하면 너무 창피하지 않을까요?,창피할 게 뭐가 있어요. 한 마디도 못 해보고 혼자 끙끙 앓다가 후회하는 사람들도 ...,2
400,이게 뭐가 부러워요. 저는 괜한 짓을 한 게 아닌가 후회하고 있는데요.,그렇지 않아요. 용기 덕분에 네 사랑을 쟁취할 수도 있잖아요. 아마 사수도 진지하게...,0


In [5]:
BOS = '</s>'
EOS = '</s>'
PAD = '<pad>'
UNK = '<unk>'
MASK = '<unused0>'
SENT = '<unused1>' # 질문/답변 구분
Q_TKN = '<usr>' # 질문
A_TKN = '<sys>' # 답변

tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2",
        bos_token=BOS, eos_token=EOS, unk_token=UNK, pad_token=PAD, mask_token=MASK,)

In [6]:
tokenizer.tokenize("안녕하세요. 공감에 특화된 챗봇을 만드는 중입니다.")

['▁안녕',
 '하',
 '세',
 '요.',
 '▁공',
 '감에',
 '▁특',
 '화된',
 '▁',
 '챗',
 '봇',
 '을',
 '▁만드는',
 '▁중',
 '입니다.']

In [7]:
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=50):
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self._data)

    def __getitem__(self, idx):
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 질문의 구두점 제거

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 답변의 구두점 제거

        q_tokenized = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_tokenized)

        a_tokenized = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_tokenized)

        # 질문 길이가 max_len보다 긴 경우
        if q_len > self.max_len:
            a_len = self.max_len - q_len
            if a_len <= 0:
                q_tokenized = q_tokenized[-(int(self.max_len / 2)) :]   # 질문 길이 = max_len의 반으로
                q_len = len(q_tokenized)
                a_len = self.max_len - q_len              # 답변 길이 = max_len - 질문 길이
            a_tokenized = a_tokenized[:a_len]
            a_len = len(a_tokenized)

        # (질문 길이 + 답변 길이)가 max_len보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_tokenized = q_tokenized[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_tokenized)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_tokenized = a_tokenized[:a_len]
            a_len = len(a_tokenized)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_tokenized[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)

        # 답변 labels를 index로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # (질문 + 답변)을 index로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_tokenized + a_tokenized)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        # (질문 + 답변), 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)


def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

# 사전학습된 KoGPT2 모델 로드
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

"""
# 사전학습된 koGPT2 테스트해보기
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=50,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)
"""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dataset = ChatbotDataset(data, max_len=50)

# Window 환경에서 num_workers=0으로 지정, Linux에서는 2로 지정
train_dataloader = DataLoader(train_dataset, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)


# 파라미터 설정
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=3e-5) # 0.00003

num_epoch = 10
Sneg = -1e18 # 모델 손실 최소화를 위해 설정


# 모델 학습
model.to(device)
model.train()

print("Training start...")
for epoch in range(num_epoch):
    total_loss = 0.0  # 각 에포크의 총 손실을 추적하기 위한 변수
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        token_ids = token_ids.to(device) # 디바이스 문제로 추가
        mask = mask.to(device) # 디바이스 문제로 추가
        label = label.to(device) # 디바이스 문제로 추가
        out = model(token_ids)
        out = out.logits  # 입력 요소의 로짓(logit)으로 구성된 새로운 텐서 반환
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()

        total_loss += avg_loss.item()  # 현재 배치의 평균 손실을 총 손실에 더함

    # 한 에포크 종료 후 평균 손실 출력
    print(f"Epoch [{epoch + 1}/{num_epoch}] - Average Loss: {total_loss / len(train_dataloader)}")

print("Training end!!")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Best model saved at: 
Training start...


<ipython-input-7-8427e21a9e7c>:75: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch [1/10] - Average Loss: 23.841907501220703
Epoch [2/10] - Average Loss: 22.8975469149076
Epoch [3/10] - Average Loss: 21.950473051804764
Epoch [4/10] - Average Loss: 21.501738181480995
Epoch [5/10] - Average Loss: 20.848711013793945
Epoch [6/10] - Average Loss: 20.49918923011193
Epoch [7/10] - Average Loss: 20.580744083111103
Epoch [8/10] - Average Loss: 19.95314935537485
Epoch [9/10] - Average Loss: 19.835338812607986
Epoch [10/10] - Average Loss: 19.615193880521336
Training end!!


In [19]:
# 학습 완료된 챗봇 테스트

# import torch
# from transformers import GPT2LMHeadModel, GPT2Tokenizer

# model_name = 'skt/kogpt2-base-v2'  # 모델 이름 (본인이 학습한 모델 이름)
# model = GPT2LMHeadModel.from_pretrained(model_name)
# tokenizer = GPT2Tokenizer.from_pretrained(model_name)

while True:
    q = input("user > ").strip()
    if q == "quit":
        break
    a = ""
    while True:
        input_ids = torch.LongTensor(tokenizer.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
        pred = model(input_ids.to(device))
        pred = pred.logits
        #gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
        gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred.cpu(), dim=-1).squeeze().numpy().tolist())[-1]
        if gen == EOS:
            break
        a += gen.replace("▁", " ")
    print("Chatbot > {}".format(a.strip()))


user > 나 오늘 우울해서 머리를 잘랐어
Chatbot > 왜 안 좋은데  무슨 일 있어
user > 다음주에 시험인데 어떡하지?
Chatbot > 응  시험까지 한 달이나 남았을 텐데 왜 그러냐  하루 만에 일 쳐내는 건 아닐 거야
user > 요즘 너무 피곤해서 걱정이야
Chatbot > 네가 지금 많이 피곤해서 고생 많다고 들었는데  무슨 일 있었어
user > 건강을 좀 챙겨야겠지?
Chatbot > 그래  다음에 보양식이라도 챙겨야겠어
user > quit
